# Day 2: PyTorch 核心模块详解

本 Notebook 将详细介绍 PyTorch 中 `torch.nn` 和 `torch.optim` 模块的核心组件。我们将深入探讨各个模块的参数含义、使用方法以及应用场景。

## 目录
1. [nn.Linear (全连接层)](#1-nnlinear-全连接层)
2. [nn.ConvNd & Pooling (卷积与池化)](#2-nnconvnd--pooling-卷积与池化)
3. [Normalization & Regularization (归一化与正则化)](#3-normalization--regularization-归一化与正则化)
4. [Containers (容器)](#4-containers-容器)
5. [nn.Loss (损失函数)](#5-nnloss-损失函数)
6. [nn.Optim (优化器)](#6-nnoptim-优化器)
7. [nn.functional (函数式接口)](#7-nnfunctional-函数式接口)

## 1. nn.Linear (全连接层)

全连接层（Fully Connected Layer）是最基础的神经网络层，它对输入数据进行线性变换：$y = xW^T + b$。

### 核心参数
- `in_features`: 输入特征的大小（输入的最后一个维度）。
- `out_features`: 输出特征的大小。
- `bias`: 是否使用偏置 $b$，默认为 `True`。

### 代码示例

In [1]:
import torch
import torch.nn as nn

# 定义一个全连接层：输入维度 10 -> 输出维度 5
linear = nn.Linear(in_features=10, out_features=5)

# 模拟输入数据：Batch Size = 2, Feature Size = 10
input_tensor = torch.randn(2, 10)

# 前向传播
output = linear(input_tensor)

print("Input shape:", input_tensor.shape)   # torch.Size([2, 10])
print("Output shape:", output.shape)        # torch.Size([2, 5])
print("Weight shape:", linear.weight.shape) # torch.Size([5, 10])
print("Bias shape:", linear.bias.shape)     # torch.Size([5])

Input shape: torch.Size([2, 10])
Output shape: torch.Size([2, 5])
Weight shape: torch.Size([5, 10])
Bias shape: torch.Size([5])


## 2. nn.ConvNd & Pooling (卷积与池化)

卷积神经网络（CNN）的核心组件。最常用的是 `nn.Conv2d`（用于图像）。

### 2.1 nn.Conv2d (二维卷积)

用于处理图像数据（Batch, Channel, Height, Width）。

#### 核心参数详解
- `in_channels`: 输入图像的通道数（例如 RGB 图像为 3）。
- `out_channels`: 卷积产生的通道数（即卷积核的数量）。
- `kernel_size`: 卷积核的大小。可以是整数（如 3 表示 3x3）或元组（如 (3, 5)）。
- `stride`: 步长。控制卷积核滑动的步距。默认为 1。
- `padding`: 填充。在输入周围填充 0 的层数。常用于保持输出尺寸不变（padding = kernel_size // 2）。
- `dilation`: 空洞卷积的间距。用于增加感受野。
- `groups`: 分组卷积。默认为 1。设为 `in_channels` 时为深度可分离卷积（Depthwise Conv）。

#### 输出尺寸公式
$$ H_{out} = \lfloor \frac{H_{in} + 2 \times \text{padding}[0] - \text{dilation}[0] \times (\text{kernel\_size}[0] - 1) - 1}{\text{stride}[0]} + 1 \rfloor $$

### 代码示例

In [2]:
# 定义一个卷积层
# 输入通道 3 (RGB), 输出通道 16, 卷积核 3x3, 步长 1, 填充 1
conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)

# 模拟输入数据：Batch=1, Channel=3, Height=32, Width=32
input_img = torch.randn(1, 3, 32, 32)

output_img = conv(input_img)

print("Input shape:", input_img.shape)   # torch.Size([1, 3, 32, 32])
print("Output shape:", output_img.shape) # torch.Size([1, 16, 32, 32]) (尺寸不变，通道变16)

Input shape: torch.Size([1, 3, 32, 32])
Output shape: torch.Size([1, 16, 32, 32])


### 2.2 Pooling (池化层)

用于下采样，减少特征图尺寸，降低计算量。

- **`nn.MaxPool2d`**: 最大池化。取窗口内的最大值，保留最显著特征。
- **`nn.AvgPool2d`**: 平均池化。取窗口内的平均值，平滑特征。

#### 常用参数
- `kernel_size`: 池化窗口大小（如 2 表示 2x2）。
- `stride`: 步长，默认等于 `kernel_size`（即不重叠）。

In [3]:
pool = nn.MaxPool2d(kernel_size=2, stride=2)

# 输入：[1, 16, 32, 32]
pooled_output = pool(output_img)

print("Pooled shape:", pooled_output.shape) # torch.Size([1, 16, 16, 16]) (尺寸减半)

Pooled shape: torch.Size([1, 16, 16, 16])


## 3. Normalization & Regularization (归一化与正则化)

### 3.1 nn.BatchNorm2d (批归一化)

在卷积层之后使用，加速收敛并防止过拟合。它在 Batch 维度上进行归一化。

- **`num_features`**: 输入的通道数（`C`）。
- **注意**: BN 层在 `train()` 和 `eval()` 模式下行为不同。
  - `train()`: 使用当前 Batch 的均值和方差，并更新全局运行均值/方差。
  - `eval()`: 使用全局运行均值和方差。

### 3.2 nn.Dropout (随机失活)

在训练过程中以概率 `p` 随机将输入张量中的元素置为 0，用于防止过拟合。

- **`p`**: 丢弃概率（默认为 0.5）。
- **注意**: 仅在 `train()` 模式下生效，在 `eval()` 模式下不做任何操作。

In [4]:
# BatchNorm 示例
bn = nn.BatchNorm2d(num_features=16)
bn_output = bn(pooled_output)
print("BN output shape:", bn_output.shape)

# Dropout 示例
dropout = nn.Dropout(p=0.5)
x = torch.randn(5, 10)

# 训练模式
dropout.train()
print("Train mode (some zeros):\n", dropout(x))

# 测试模式
dropout.eval()
print("Eval mode (no zeros):\n", dropout(x))

BN output shape: torch.Size([1, 16, 16, 16])
Train mode (some zeros):
 tensor([[ 0.0000, -0.0000, -0.2206, -2.7172, -1.2545, -0.0000,  0.1108, -1.7645,
         -1.0419, -0.1367],
        [ 0.0000,  0.8157, -0.0000,  0.0000, -1.3677, -0.0000,  0.0000, -0.0000,
          0.0000,  2.0330],
        [-0.0000, -0.0000, -0.4552, -0.0000,  0.4040,  1.6006, -0.0000, -0.0000,
          0.0000,  0.0000],
        [-1.9678, -0.5675, -0.6396,  0.4800, -0.0000, -0.1160,  4.4152, -0.0000,
          0.0000,  0.0000],
        [ 0.0000,  1.7427, -2.1221,  0.0000,  1.2788, -0.0000,  0.0000, -0.6074,
         -0.0000, -0.0000]])
Eval mode (no zeros):
 tensor([[ 0.1692, -0.1750, -0.1103, -1.3586, -0.6272, -0.9493,  0.0554, -0.8822,
         -0.5209, -0.0684],
        [ 0.0232,  0.4078, -1.0458,  0.1475, -0.6839, -0.1480,  0.6321, -0.1751,
          0.1135,  1.0165],
        [-0.4745, -0.5931, -0.2276, -0.4770,  0.2020,  0.8003, -1.5625, -0.3660,
          1.5857,  0.0419],
        [-0.9839, -0.2837, -0.319

## 4. Containers (容器)

用于组合多个网络层。

### 4.1 nn.Sequential

按顺序将模块添加到容器中，数据会按顺序通过这些模块。这是构建简单前馈网络最快的方式。

### 4.2 nn.ModuleList

像 Python 的 `list` 一样存储 `nn.Module`，但会自动注册参数。**注意**：`ModuleList` 不会自动定义前向传播逻辑，你需要在 `forward` 函数中手动遍历调用。

In [5]:
# 使用 Sequential 快速搭建一个小网络
model = nn.Sequential(
    nn.Conv2d(3, 16, 3, 1, 1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(2)
)

print(model)
# output = model(input_img)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


## 5. nn.Loss (损失函数)

损失函数衡量模型输出与真实标签之间的差异。

### 5.1 nn.MSELoss (均方误差损失)
- **用途**: 回归问题（预测连续值）。
- **公式**: $L = \frac{1}{N} \sum (y_{pred} - y_{true})^2$

### 5.2 nn.CrossEntropyLoss (交叉熵损失)
- **用途**: 多分类问题。
- **输入**: 模型输出的 Logits（**未**经过 Softmax 的原始分数）和 类别索引（LongTensor）。
- **注意**: 该函数内部已经包含了 `LogSoftmax` 和 `NLLLoss`，所以模型末层**不需要**加 Softmax。

### 5.3 nn.BCEWithLogitsLoss
- **用途**: 二分类问题。
- **输入**: Logits。
- **注意**: 内部包含了 Sigmoid，比手动 Sigmoid + BCELoss 数值更稳定。

In [6]:
# CrossEntropyLoss 示例
criterion = nn.CrossEntropyLoss()

# 假设 Batch=2, 3个类别
logits = torch.randn(2, 3) # 模型输出
targets = torch.tensor([0, 2]) # 真实标签

loss = criterion(logits, targets)
print("Loss:", loss.item())

Loss: 1.4665141105651855


## 6. nn.Optim (优化器)

优化器用于根据梯度更新模型参数。

### 常用优化器
- **`optim.SGD`**: 随机梯度下降。常用参数 `lr` (学习率), `momentum` (动量)。
- **`optim.Adam`**: 自适应矩估计。通常收敛更快。常用参数 `lr`。

### 标准训练步骤
1. `optimizer.zero_grad()`: 清空过往梯度。
2. `loss.backward()`: 反向传播计算当前梯度。
3. `optimizer.step()`: 根据梯度更新参数。

In [7]:
import torch.optim as optim

# 定义优化器，传入需要更新的参数
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 模拟训练步
optimizer.zero_grad() # 1. 清零
# loss = criterion(output, target) # 计算loss
# loss.backward() # 2. 反向传播
optimizer.step() # 3. 更新参数

## 7. nn.functional (函数式接口)

`torch.nn.functional` (通常导入为 `F`) 包含了与 `nn.Module` 对应的无状态函数。

### 何时使用？
- 如果层**有可学习参数**（如 Linear, Conv2d, BatchNorm），使用 `nn.Module`。
- 如果层**没有参数**（如 ReLU, MaxPool, Softmax），既可以用 `nn.Module` 也可以用 `F`。通常在 `forward` 函数中直接调用 `F` 更简洁。

### 常用函数
- `F.relu(x)`
- `F.softmax(x, dim=1)`
- `F.interpolate(x, scale_factor=2, mode='bilinear')`: 上采样/调整图像大小。
- `F.pad(x, pad)`: 填充操作。

In [8]:
import torch.nn.functional as F

class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        # 有参数的层使用 self.layer
        x = self.conv1(x)
        # 无参数的操作可以直接使用 F.relu
        x = F.relu(x)
        return x